In [99]:
#####Latent Dirichlet Allocation Model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

####データの発生####
##データの設定
k = 15   #トピック数
d = 3000   #文書数
v  = 1000   #語彙数
w = poisson(175, d)   #1文書あたりの単語数
f = sum(w)   #総単語数

In [100]:
##パラメータの設定
#ディレクリ分布のパラメータの設定
alpha0 = np.repeat(0.2, k)   #文書のディレリ事前分布のパラメータ
alpha1 = np.repeat(0.15, v)   #単語のディレクリ事前分布のパラメータ

#ディレクリ分布からパラメータを発生
thetat = theta = np.random.dirichlet(alpha0, d)   #文書のトピック分布をディレクリ乱数から発生
phit = phi = np.random.dirichlet(alpha1, k)    #単語のトピック分布をデレクレリ乱数から発生

In [101]:
##多項分布より単語頻度行列を生成
WX = np.zeros((d, v), dtype='int')
Z = [i for i in range(d)]
vec = np.arange(0, k)

for i in range(d):
    #文書のトピックを生成
    z = multinomial(1, theta[i, :], w[i])   #文書のトピック分布を発生
    z_vec = np.dot(z, vec)

    #トピック割当から単語を生成
    word = np.zeros((w[i], v))
    for j in range(w[i]):
        word[j, :] = multinomial(1, phi[z_vec[j], :], 1)

    WX[i, :] = np.sum(word, axis=0)
    Z[i] = z_vec

In [102]:
####トピックモデルのためのデータと関数の準備####
##それぞれの文書中の単語の出現をベクトルに並べる
##データ推定用IDを作成
ID_list = [i for i in range(d)]
wd_list = [i for i in range(d)]

#文書ごとに求人IDおよび単語IDを作成
for i in range(d):
    ID_list[i] = np.repeat(i, w[i])
    num1 = (WX[i, :] > 0) * np.arange(1, v+1)
    num2 = num1[num1!=0]
    W1 = WX[i, WX[i, :] > 0]
    number = np.repeat(num2, W1)
    wd_list[i] = number

#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ID_d = np.zeros(sum(w), dtype='int')
start = 0

for i in range(d):
    wd[start:start+w[i]] = wd_list[i] - 1
    ID_d[start:start+w[i]] = ID_list[i]
    start += w[i] 

In [103]:
##インデックスを作成
doc_list = [i for i in range(d)]
doc_vec = [i for i in range(d)]
word_list = [i for i in range(v)]
word_vec = [i for i in range(v)]
index = np.array(range(f))

for i in range(d):
    doc_list[i] = index[ID_d==i]
    doc_vec[i] = np.repeat(1, doc_list[i].shape)
for i in range(v):
    word_list[i] = index[wd==i]
    word_vec[i] = np.repeat(1, word_list[i].shape)

In [104]:
####マルコフ連鎖モンテカルロ法でLDAのパラメータをサンプリング####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w, k):
    Bur = np.zeros((f, k))
    Bur0 = np.zeros((f, k))

    #負担係数を計算
    for j in range(k):
        Bi = np.repeat(theta[:, j], w) * phi[j, wd]
        Bur[:, j] = Bi

    #負担率の分母部分
    Bur_sums = np.sum(Bur, axis=1)
    for l in range(k):
        Bur0[:, l] = Bur_sums 

    #負担率と混合率を計算
    Br = Bur / Bur0   #負担率の計算
    r = np.sum(Br, axis=0) / np.sum(Br)   #混合率の計算
    return Br, Bur, r

In [105]:
##アルゴリズムの設定
#更新ステータス
LLo = -100000000
LLw = LLo
iter = 1
dl = 100   #EMステップでの対数尤度の差の初期値
tol = 0.5
abs(dl)

100

In [106]:
##事前分布の設定
alpha1 = 0.1
beta1 = 0.1

##パラメータの初期値
mu = np.sum(WX, axis=0)/np.sum(WX)
phi = np.random.dirichlet(np.repeat(1, v), k)
theta = np.random.dirichlet(np.repeat(1, k), d)

In [107]:
##LDAのパラメータを更新
while abs(dl) >= tol:

    ##単語トピックを生成
    #トピック割当の潜在確率の期待値を推定
    out = burden_fr(f, theta, phi, wd, w, k)
    Zi = out[0]
    Zi_T = Zi.T

    ##パラメータを更新
    #トピック分布thetaの期待値を推定
    wsum = np.zeros((d, k))
    for i in range(d):
        wsum[i :] = np.dot(Zi_T[:, doc_list[i]], doc_vec[i]) + alpha1   #ディクレリ分布のパラメータ
    theta = wsum / np.repeat(np.sum(wsum, axis=1), k).reshape((d, k))   #ディクレリ分布の期待値

    #単語分布phiの期待値を推定
    vsum = np.zeros((k, v))
    for j in range(v):
        vsum[:, j] = np.dot(Zi_T[:, word_list[j]], word_vec[j]) + beta1   #ディクレリ分布のパラメータ
    phi = vsum / np.repeat(np.sum(vsum, axis=1), v).reshape((k, v))   #ディクレリ分布の期待値

    #対数尤度の更新
    LLS = np.sum(np.log(np.sum(out[1], axis=1)))   #対数尤度を計算
    iter <- iter+1
    dl = LLS-LLo
    LLo = LLS
    LLw <- np.array((LLw, LLo))
    print(LLo)

-3665206.26141573
-3524012.437995131
-3519233.920428751
-3515632.0665636454
-3512456.7160359453
-3509269.33724384
-3505735.167697679
-3501546.2353034844
-3496401.8574330574
-3490028.135557488
-3482225.2924322924
-3472919.214200634
-3462187.9761682176
-3450250.146099597
-3437423.3077268815
-3424080.4669471183
-3410613.1429871316
-3397393.4658924793
-3384749.571807621
-3372956.14327563
-3362235.5828055213
-3352751.458572164
-3344574.601670342
-3337657.9102787827
-3331867.502682771
-3327026.8813327895
-3322937.0446093795
-3319394.276545844
-3316220.0707886657
-3313296.887347737
-3310591.30624262
-3308144.6300520613
-3306027.684177588
-3304284.9520807457
-3302907.4607026908
-3301842.615025084
-3301022.2880715183
-3300385.677044532
-3299884.861943717
-3299483.193866447
-3299154.338286046
-3298880.5987077844
-3298650.1728059943
-3298454.9452267042
-3298288.969171941
-3298147.4204942295
-3298026.15092923
-3297921.631215465
-3297830.9472356164
-3297751.734800854
-3297682.0823193225
-3297620.43